# Part 1

Suddenly, a scheduled job activates the system's disk defragmenter. Were the situation different, you might sit and watch it for a while, but today, you just don't have that kind of time. It's soaking up valuable system resources that are needed elsewhere, and so the only option is to help it finish its task as soon as possible.

The disk in question consists of a 128x128 grid; each square of the grid is either free or used. On this disk, the state of the grid is tracked by the bits in a sequence of knot hashes.

A total of 128 knot hashes are calculated, each corresponding to a single row in the grid; each hash contains 128 bits which correspond to individual grid squares. Each bit of a hash indicates whether that square is free (0) or used (1).

The hash inputs are a key string (your puzzle input), a dash, and a number from 0 to 127 corresponding to the row. For example, if your key string were `flqrgnkx`, then the first row would be given by the bits of the knot hash of `flqrgnkx-0`, the second row from the bits of the knot hash of `flqrgnkx-1`, and so on until the last row, `flqrgnkx-127`.

The output of a knot hash is traditionally represented by 32 hexadecimal digits; each of these digits correspond to 4 bits, for a total of `4 * 32 = 128` bits. To convert to bits, turn each hexadecimal digit to its equivalent binary value, high-bit first: 0 becomes 0000, 1 becomes 0001, e becomes 1110, f becomes 1111, and so on; a hash that begins with `a0c2017...` in hexadecimal would begin with `10100000110000100000000101110000...` in binary.

Continuing this process, the first 8 rows and columns for key `flqrgnkx` appear as follows, using # to denote used squares, and . to denote free ones:

```
##.#.#..-->
.#.#.#.#   
....#.#.   
#.#.##.#   
.##.#...   
##..#..#   
.#...#..   
##.#.##.-->
|      |   
V      V   
```

In this example, 8108 squares are used across the entire 128x128 grid.

Given your actual key string, how many squares are used?

Your puzzle input is `amgozmfv`.

In [1]:
from functools import reduce
from itertools import chain
from operator import xor

Next three cells copied from Day 10 exercise.

In [2]:
def pzl_input(input_):
    return [ord(c) for c in input_] + [17, 31, 73, 47, 23]

In [3]:
def hasher(list_: list, lengths: list, pos=0, skip_size=0):
    list_len = len(list_)
    
    def _(x):
        return None if x < 0 else x
    
    for len_ in lengths:
        if len_ <= 1:
            pass
        elif (pos + len_) < list_len:
            # no wrapping
            fslice = slice(pos, pos + len_)
            rslice = slice(_(pos + len_ - 1), _(pos - 1), -1)
            list_[fslice] = list_[rslice]
        else:
            # wrapping
            end_slice = slice(pos, pos + len_)
            front_slice = slice(0, len_ - (list_len - pos))
            selected = (list_[end_slice] + list_[front_slice])[::-1]
            list_[end_slice] = selected[:(list_len - pos)]
            list_[front_slice] = selected[(list_len - pos):]
        
        pos = (pos + len_ + skip_size) % list_len
        skip_size += 1

    return list_, pos, skip_size

In [4]:
def knot_hash(lengths):
    pos = 0
    skip_size = 0
    list_ = list(range(256))
    
    for _ in range(64):
        _, pos, skip_size = hasher(list_, lengths, pos, skip_size)

    return ''.join(
        hex(reduce(xor, list_[i:i + 16]))[2:].zfill(2)
        for i in range(0, 256, 16))

In [5]:
bin(int('3', 16))

'0b11'

In [6]:
def hex_to_binary(hex_str):
    yield from chain.from_iterable(bin(int(c, 16))[2:].zfill(4) for c in hex_str)

In [44]:
assert ''.join(hex_to_binary('a0c20170')) == '10100000110000100000000101110000', ''.join(hex_to_binary('a0c2017'))

In [43]:
assert len(''.join(hex_to_binary(knot_hash(pzl_input('flqrgnkx-2'))))) == 128

In [42]:
assert ''.join(hex_to_binary(knot_hash(pzl_input('flqrgnkx-0'))))[:8] == '11010100'
assert ''.join(hex_to_binary(knot_hash(pzl_input('flqrgnkx-1'))))[:8] == '01010101'
assert ''.join(hex_to_binary(knot_hash(pzl_input('flqrgnkx-2'))))[:8] == '00001010'

In [45]:
rows = ('flqrgnkx-' + str(i) for i in range(128))
result = sum(int(x) for row in rows for x in hex_to_binary(knot_hash(pzl_input(row))))
assert  result == 8108, result

In [46]:
rows = ('amgozmfv-' + str(i) for i in range(128))
sum(int(x) for row in rows for x in hex_to_binary(knot_hash(pzl_input(row))))

8222

# Part 2

Now, all the defragmenter needs to know is the number of regions. A region is a group of used squares that are all adjacent, not including diagonals. Every used square is in exactly one region: lone used squares form their own isolated regions, while several adjacent squares all count as a single region.

In the example above, the following nine regions are visible, each marked with a distinct digit:

```
11.2.3..-->
.1.2.3.4   
....5.6.   
7.8.55.9   
.88.5...   
88..5..8   
.8...8..   
88.8.88.-->
|      |   
V      V   
```

Of particular interest is the region marked 8; while it does not appear contiguous in this small view, all of the squares marked 8 are connected when considering the whole 128x128 grid. In total, in this example, 1242 regions are present.

How many regions are present given your key string?

In [7]:
import itertools

import numpy as np

In [8]:
rows = ('flqrgnkx-' + str(i) for i in range(128))
arr = np.int8([list(hex_to_binary(knot_hash(pzl_input(row)))) for row in rows])
arr

array([[1, 1, 0, ..., 1, 1, 0],
       [0, 1, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 0, 1],
       ..., 
       [1, 1, 1, ..., 1, 0, 0],
       [1, 1, 0, ..., 1, 0, 1],
       [0, 0, 1, ..., 1, 1, 0]], dtype=int8)

In [11]:
arr.shape

(128, 128)

In [9]:
def in_bounds(shape, point):
    if 0 <= point[0] < shape[0] and 0 <= point[1] < shape[1]:
        return True
    else:
        return False

In [16]:
assert in_bounds((10, 10), (0, 0)) is True
assert in_bounds((10, 10), (-1, 0)) is False
assert in_bounds((10, 10), (10, 0)) is False
assert in_bounds((10, 10), (10, 10)) is False
assert in_bounds((10, 10), (6, 7)) is True

In [10]:
def connect_neighbors(grid, segments, pos, segment_no=None):
    segment_no = segment_no or (max(segments.values(), default=0) + 1)
    segments[pos] = segment_no
    for dir_ in ((1, 0), (0, 1), (-1, 0), (0, -1)):
        neighbor = (pos[0] + dir_[0], pos[1] + dir_[1])
        if neighbor not in segments and in_bounds(grid.shape, neighbor) and grid[neighbor] == 1:
            connect_neighbors(grid, segments, neighbor, segment_no)

In [11]:
def segment(grid: np.array) -> dict:
    nrows, ncols = grid.shape
    segments = {}
    for pos in itertools.product(range(nrows), range(ncols)):
        if grid[pos] == 1 and pos not in segments:
            connect_neighbors(grid, segments, pos)
    return segments

In [14]:
def segments_to_array(segments: dict, shape: np.array) -> np.array:
    grid = np.zeros(shape, dtype=np.int_)
    for pos, seg in segments.items():
        grid[pos] = seg
    return grid

Validating the test case. The numbers come out different because of the bigger grid, but it should look the same as the test case above in terms of segmentation.

In [16]:
result = segments_to_array(segment(arr), arr.shape)

In [17]:
result[:8, :8]

array([[ 1,  1,  0,  2,  0,  3,  0,  0],
       [ 0,  1,  0,  2,  0,  3,  0,  4],
       [ 0,  0,  0,  0, 30,  0, 31,  0],
       [38,  0,  5,  0, 30, 30,  0, 39],
       [ 0,  5,  5,  0, 30,  0,  0,  0],
       [ 5,  5,  0,  0, 30,  0,  0,  5],
       [ 0,  5,  0,  0,  0,  5,  0,  0],
       [ 5,  5,  0,  5,  0,  5,  5,  0]])

In [19]:
assert result.max() == 1242

In [18]:
rows = ('amgozmfv-' + str(i) for i in range(128))
arr = np.int8([list(hex_to_binary(knot_hash(pzl_input(row)))) for row in rows])
segments = segment(arr)
max(segments.values())

1086